In [ ]:
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

def get_hyperlinks(body):
    import os 
    os.system("pip install beautifulsoup4")
    from bs4 import BeautifulSoup
    output = []
    soup = BeautifulSoup(body)
    a_s = soup.find_all('a')
    a_s = list(filter(lambda x: x.attrs.get('href'), a_s))
    
    output = []
    for a in a_s:
        attrs = a.attrs
        attrs['text'] = a.get_text().strip()
        output.append(attrs)
    
    return output
    
get_hyperlinks_udf = udf(f=get_hyperlinks, returnType=ArrayType(
    StructType([
        StructField('href', StringType()),
        StructField('title', StringType()),
        StructField('itemscope', StringType()),
        StructField('itemprop', StringType()),
        StructField('text', StringType()),
    ])
))

full_beat_df = (
    newsarchive_df
     .filter(newsarchive_df.wire == 25)
     .filter(check_overlap_udf(newsarchive_df.nicodes))
)

hyperlinks_df = (
    newsarchive_df
     .filter(newsarchive_df.wire == 25)
     .filter(check_overlap_udf(newsarchive_df.nicodes))
     .limit(2000)
     .select(get_hyperlinks_udf('body').alias('hyperlinks'))
     .toPandas()
)